In [6]:
import json
import pandas as pd
from datetime import datetime

with open('D:\OneDrive\Desktop\MCA\Python\ChatGPT\conversations.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

rows = []

for convo in data:
    title = convo.get('title', 'No Title')
    created_ts = convo.get('create_time', None)
    if created_ts:
        created = datetime.fromtimestamp(created_ts)
    else:
        created = None

    mapping = convo.get("mapping", {})
    for node_id, node in mapping.items():
        message = node.get('message')
        if message is None:
            continue  # Skip if no message

        author = message.get('author')
        role = author.get('role') if author else 'system'

        parts = message.get('content', {}).get('parts')
        if parts and isinstance(parts, list):
            content = parts[0]
        else:
            content = ''

        rows.append({
            "title": title,
            "datetime": created,
            "role": role,
            "message": content,
            "length": len(content)
        })

# Save to CSV
df = pd.DataFrame(rows)
df.to_csv("chat_data.csv", index=False)
print("Exported chat_data.csv successfully.")

Exported chat_data.csv successfully.


<>:5: SyntaxWarning:

invalid escape sequence '\O'

<>:5: SyntaxWarning:

invalid escape sequence '\O'

C:\Users\Abrarali\AppData\Local\Temp\ipykernel_5292\4000322456.py:5: SyntaxWarning:

invalid escape sequence '\O'



In [7]:
import plotly.express as px

df['role'] = df['role'].map({'user': 'User', 'assistant': 'ChatGPT'})

fig = px.scatter_3d(
    df,
    x='datetime',
    y='title',
    z='length',
    color='role',
    hover_data=['message']
)

# fig.show()


In [8]:
import plotly.io as pio
pio.renderers.default = "browser"
fig.show()

In [9]:
from PIL import Image
import os

image_data = []

for file in os.listdir("images"):
    if file.endswith(".png"):
        path = os.path.join("images", file)
        img = Image.open(path)
        width, height = img.size
        image_data.append({"filename": file, "width": width, "height": height})

image_df = pd.DataFrame(image_data)


In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap
import plotly.express as px

# Load your processed chat data
df = pd.read_csv("chat_data.csv")

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get 384-dim embeddings for each message
embeddings = model.encode(df['message'].fillna("").tolist())

# Reduce to 3D using UMAP
reducer = umap.UMAP(n_components=3, random_state=42)
embedding_3d = reducer.fit_transform(embeddings)

# Add to dataframe
df['x'] = embedding_3d[:, 0]
df['y'] = embedding_3d[:, 1]
df['z'] = embedding_3d[:, 2]


c:\Users\Abrarali\AppData\Local\Programs\Python\Python313\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [11]:
fig = px.scatter_3d(
    df,
    x='x',
    y='y',
    z='z',
    color='role',
    hover_data=['title', 'message', 'datetime'],
    opacity=0.7,
    title='🧠 Your 3D Thinking Map'
)

fig.update_traces(marker=dict(size=5))
fig.show()


In [12]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0)
df['topic_cluster'] = kmeans.fit_predict(embeddings)
color='topic_cluster'  # for Plotly
